In [1]:
import boto3
import json
from botocore.client import Config


In [2]:
knowledgeBaseId = '4V4UV8DXOX'
model_id ='anthropic.claude-3-haiku-20240307-v1:0'

bedrock_config = Config(connect_timeout=120, 
                        read_timeout=120, 
                        retries={'max_attempts': 0}
                       )
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config)

In [17]:
router_prompt = '''You are a expert in identifying intent from the user question.  
You will read the text provided carefully and determine the intent of the user.  You will then classify the intent in to one of 2 categories - coding_assistant or  architecture_assistant.  
If user request is for providing coding help then categorize it as coding_assistant.  If user request is for providing guidance, best practices, architecture questions calssify them  as architecture_assistant
Respond with just the classification type.  Do NOT add any preamble.'''

In [18]:
user_query = 'what are the 5 pillars of well architected framework'

In [19]:
final_prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{router_prompt}: {user_query}"
                    }
                ]
            }
        ]
    }

final_prompt = json.dumps(final_prompt)
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=final_prompt,
)
response_body = json.loads(response.get('body').read())
response_body['content'][0]['text']

'architecture_assistant'

In [11]:
response_body = json.loads(response.get('body').read())
response_body['content'][0]['text']

'Coding Assistance'

In [14]:
encoder = CohereEncoder()
rl = RouteLayer(encoder=encoder, routes=routes)

2024-04-16 15:24:15 INFO semantic_router.utils.logger Initializing RouteLayer


In [15]:
def software_architect(prompt):
    relevant_documents = bedrock_agent_client.retrieve(
    retrievalQuery= {
        'text': prompt
    },
    knowledgeBaseId=knowledgeBaseId,
    retrievalConfiguration= {
        'vectorSearchConfiguration': {
            'numberOfResults': 3 # will fetch top 3 documents which matches closely with the query.
        }
    }
    )

    context = ''
    for doc in relevant_documents['retrievalResults']:
        context += doc['content']['text']
        
    final_prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt}: {context}"
                    }
                ]
            }
        ]
    }
    final_prompt = json.dumps(final_prompt)
    streaming_response = bedrock_client.invoke_model_with_response_stream(
        modelId=model_id,
        body=final_prompt,
    )
    stream = streaming_response.get('body')

    # a little bit of magic to buffer some response chunks then print them
    acc = 0
    buff = ''
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                blob = json.loads(chunk.get('bytes').decode())
                if blob['type'] == 'content_block_delta':
                    buff = buff + blob['delta']['text']
                    if acc < 20:
                        acc = acc +1
                    else:
                        print(buff)
                        buff=''
                        acc = 0

In [16]:
def coding_assistant(prompt):
    final_prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt}"
                    }
                ]
            }
        ]
    }
    final_prompt = json.dumps(final_prompt)
    streaming_response = bedrock_client.invoke_model_with_response_stream(
        modelId=model_id,
        body=final_prompt,
    )
    
    stream = streaming_response.get('body')

    # a little bit of magic to buffer some response chunks then print them
    acc = 0
    buff = ''
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                blob = json.loads(chunk.get('bytes').decode())
                if blob['type'] == 'content_block_delta':
                    buff = buff + blob['delta']['text']
                    if acc < 20:
                        acc = acc +1
                    else:
                        print(buff)
                        buff=''
                        acc = 0

In [18]:
prompt = 'pillars of well architected framework'
router = rl(prompt).name

if router == "software_architect":
    software_architect(prompt)
elif router == "coding_assistant":
    coding_assistant(prompt)

The key points from the passage are:

1. The AWS Well-Architected Framework is a set of
 best practices and guidelines developed by the AWS community of principal engineers to help customers build well-architected systems
 in the cloud.

2. The framework identifies 5 pillars of a well-architected system
: operational excellence, security, reliability, performance efficiency, and cost optimization.

3. The framework provides general
 design principles such as "stop guessing capacity needs" to help guide good design in the cloud.

4
. The framework is implemented through a structured review process where teams can assess their architectures against the framework's
 guidelines.

5. By carrying out multiple reviews across an organization, common thematic issues can be identified,
 which can then be addressed through mechanisms, training, or sharing of principal engineering knowledge.

6. The goal
 is to help organizations build reliable, secure, efficient, and cost-effective systems in the

In [29]:
###open a file and read the contents in to a string
def read_file(filename):
    with open(filename, 'r') as f:
        return f.read()
    
schema = read_file('chinook.sql')
schema

'CREATE TABLE [Album]\n(\n    [AlbumId] INTEGER  NOT NULL,\n    [Title] NVARCHAR(160)  NOT NULL,\n    [ArtistId] INTEGER  NOT NULL,\n    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),\n    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n);\n\nCREATE TABLE [Artist]\n(\n    [ArtistId] INTEGER  NOT NULL,\n    [Name] NVARCHAR(120),\n    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])\n);\n\nCREATE TABLE [Customer]\n(\n    [CustomerId] INTEGER  NOT NULL,\n    [FirstName] NVARCHAR(40)  NOT NULL,\n    [LastName] NVARCHAR(20)  NOT NULL,\n    [Company] NVARCHAR(80),\n    [Address] NVARCHAR(70),\n    [City] NVARCHAR(40),\n    [State] NVARCHAR(40),\n    [Country] NVARCHAR(40),\n    [PostalCode] NVARCHAR(10),\n    [Phone] NVARCHAR(24),\n    [Fax] NVARCHAR(24),\n    [Email] NVARCHAR(60)  NOT NULL,\n    [SupportRepId] INTEGER,\n    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),\n    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] (

In [33]:
system_prompt =f'''Transform the following natural language requests into valid SQL queries. Do not add any preamble in your response.  
Assume a database with the following tables and columns exists:
{schema}
'''

In [34]:
sql_prompt = 'Get the list of customers'

In [35]:
final_prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "system" : system_prompt,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{sql_prompt}"
                    }
                ]
            }
        ]
    }

final_prompt = json.dumps(final_prompt)
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=final_prompt,
)
response_body = json.loads(response.get('body').read())
response_body['content'][0]['text']

'SELECT * FROM Customer;'

In [ ]:
knowledgeBaseId = '4V4UV8DXOX'
model_id ='anthropic.claude-3-haiku-20240307-v1:0'

In [ ]:
{
   "input": { 
      "text": "What are the 5 pillars of well architected framework"
   },
   "retrieveAndGenerateConfiguration": { 
      "knowledgeBaseConfiguration": { 
         "knowledgeBaseId": "4V4UV8DXOX",
         "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0",
         "retrievalConfiguration": { 
            "vectorSearchConfiguration": { 
               "numberOfResults": 3,
            },
         },
      },
   }
}

In [9]:
import boto3
import json

bedrock_runtime = boto3.client("bedrock-runtime")
model_Id = "anthropic.claude-3-sonnet-20240229-v1:0"
prompt = "Write code for prime number"

system_prompt = """You are a expert in identifying intent from the user question.  
                You will read the text provided carefully and determine the intent of the user.  You will then classify the intent in to one of 2 categories - coding_assistant or  architecture_assistant.  
                If user request is for providing coding help then categorize it as coding_assistant.  If user request is for providing guidance, best practices, architecture questions calssify them  as architecture_assistant
                Respond with just the classification type.  Do NOT add any preamble."""

payload = {
"modelId": model_Id,
"contentType": "application/json",
"accept": "application/json",
"body": {
    "anthropic_version": "bedrock-2023-05-31",
    "system": system_prompt,
    "max_tokens": 1000,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
}
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock_runtime.invoke_model(
    body=body_bytes,
    contentType=payload['contentType'],
    accept=payload['accept'],
    modelId=payload['modelId']
)

# Process the response
response_body = json.loads(response['body'].read())

print(response_body['content'][0])
print(response_body['content'][0]['text'])

{'type': 'text', 'text': 'coding_assistant'}
coding_assistant


In [ ]:
{
  "prompt": "write python code for prime numbers"
}